In [31]:
pip install PrettyTable # устанавливаем модуль для вывода таблиц


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
import sqlite3 as sql
import csv
from prettytable import PrettyTable

### Шаг 1

In [23]:
with sql.connect('Gubanov_transaction.db') as con:
    cur = con.cursor()
    cur.execute(f'CREATE TABLE IF NOT EXISTS transaction_bd (TX_DATETIME NUMERIC NOT NULL,CUSTOMER_ID INTEGER NOT NULL,TX_AMOUNT REAL NOT NULL)')
    

In [24]:
with open('/Users/mikhailgubanov/Downloads/final_transactions.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Пропускаем заголовок
    for row in reader:
        cur.execute('''
            INSERT INTO transaction_bd (TX_DATETIME, CUSTOMER_ID, TX_AMOUNT)
            VALUES (?, ?, ?)
        ''', (row[1], row[2], row[4])) # оставляем только столбцы указанные в задании

# Сохраняем изменения и закрываем соединение
con.commit()
con.close()

### Шаг 2

A. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);

In [39]:
def db_connect(request):
    with sql.connect('Gubanov_transaction.db') as con:
        cur = con.cursor()
        cur.execute(request)
    con.commit()
    return cur.fetchall() 

part_1_request = """SELECT CUSTOMER_ID, sum(TX_AMOUNT) as TOTAL_AMOUNT
from transaction_bd
GROUP by CUSTOMER_ID
HAVING TOTAL_AMOUNT  > 700000
order by CUSTOMER_ID ASC"""

result_1 = db_connect(part_1_request)

# Создаем таблицу и добавляем заголовки
table_1 = PrettyTable()
table_1.field_names = ["CUSTOMER_ID", "TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_1:
    table_1.add_row(row)

# Выводим таблицу
print(table_1)

+-------------+-------------------+
| CUSTOMER_ID |    TOTAL_AMOUNT   |
+-------------+-------------------+
|     389     | 753411.8999999998 |
|     2249    |     707478.64     |
|     2891    | 786115.8700000002 |
|     3116    |     721980.69     |
|     4163    | 765153.6300000006 |
+-------------+-------------------+


B. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);

In [40]:
part_2_request = """ SELECT CUSTOMER_ID, sum(TX_AMOUNT) as TOTAL_AMOUNT
from transaction_bd
WHERE TX_DATETIME BETWEEN date('2023-01-01') and date ('2023-01-13')
GROUP by CUSTOMER_ID
HAVING TOTAL_AMOUNT  > 200000
order by CUSTOMER_ID ASC """

result_2 = db_connect(part_2_request)

# Создаем таблицу и добавляем заголовки
table_2 = PrettyTable()
table_2.field_names = ["CUSTOMER_ID", "TOTAL_AMOUNT"]

# Добавляем данные в таблицу
for row in result_2:
    table_2.add_row(row)

# Выводим таблицу
print(table_2)

+-------------+--------------------+
| CUSTOMER_ID |    TOTAL_AMOUNT    |
+-------------+--------------------+
|     1918    | 209624.37999999995 |
|     3406    | 205762.23999999996 |
|     3833    |      269107.4      |
|     4252    | 208090.38000000003 |
+-------------+--------------------+


C. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;

In [41]:
part_3_request = """SELECT CUSTOMER_ID, count(TX_AMOUNT) as COUNT_Trasaction
from transaction_bd
WHERE CUSTOMER_ID LIKE '4%'
GROUP by CUSTOMER_ID
HAVING COUNT_Trasaction  > 444
order by CUSTOMER_ID ASC """ 

result_3 = db_connect(part_3_request)

# Создаем таблицу и добавляем заголовки
table_3 = PrettyTable()
table_3.field_names = ["CUSTOMER_ID", "COUNT_Trasaction"]

# Добавляем данные в таблицу
for row in result_3:
    table_3.add_row(row)

# Выводим таблицу
print(table_3)

+-------------+------------------+
| CUSTOMER_ID | COUNT_Trasaction |
+-------------+------------------+
|     4231    |       451        |
|     4539    |       445        |
|     4661    |       449        |
+-------------+------------------+


D. Создать флаг доходности клиентов по логике:
1. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
2. Если сумма транзакций больше 50000 и не более 10000 тогда вывести 'средняя доходность';
3. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.

In [48]:
# добавляем новый столбец в базу данных
part_4d1 = """ ALTER TABLE transaction_bd ADD COLUMN profitability_flg TEXT; """
db_connect(part_4d1)

OperationalError: duplicate column name: profitability_flg

In [70]:
# скрипт для проставления флага доходности 
part_4d2 = """ 
UPDATE transaction_bd as tbd
SET profitability_flg = 
	CASE 
		WHEN total_amount <= 50000 THEN 'Низкая доходность'
		WHEN total_amount > 50000 and total_amount <= 100000 THEN 'Средняя доходность'
		WHEN total_amount > 100000 THEN 'Высокая доходность'
	END
FROM 
(SELECT CUSTOMER_ID, SUM(TX_AMOUNT) as total_amount
FROM transaction_bd
GROUP BY CUSTOMER_ID
) AS t
WHERE tbd.CUSTOMER_ID = t.CUSTOMER_ID;
"""

In [71]:
# проставляем флаг доходности 
db_connect(part_4d2)

[]

E. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);

In [73]:
part_5_request = """ SELECT profitability_flg,  count (DISTINCT CUSTOMER_ID) as CNT
from transaction_bd
group by profitability_flg """

result_5 = db_connect(part_5_request)

# Создаем таблицу и добавляем заголовки
table_5 = PrettyTable()
table_5.field_names = ["profitability_flg", "CNT"]

# Добавляем данные в таблицу
for row in result_5:
    table_5.add_row(row)

# Выводим таблицу
print(table_5)

+--------------------+------+
| profitability_flg  | CNT  |
+--------------------+------+
| Высокая доходность | 2167 |
| Низкая доходность  | 1812 |
| Средняя доходность | 1007 |
+--------------------+------+


F. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).

In [75]:
part_6_request = """ SELECT date(TX_DATETIME) as 'DATE', sum(TX_AMOUNT) as Total_Ammount
from transaction_bd
GROUP by date(TX_DATETIME)
order by 1 ASC; """

result_6 = db_connect(part_6_request)

# Создаем таблицу и добавляем заголовки
table_6 = PrettyTable()
table_6.field_names = ["DATE", "TOTAL_AMMOUNT"]

# Добавляем данные в таблицу
for row in result_6:
    table_6.add_row(row)

# Выводим таблицу
print(table_6)

+------------+--------------------+
|    DATE    |   TOTAL_AMMOUNT    |
+------------+--------------------+
| 2023-01-01 | 4827656.260000017  |
| 2023-01-02 |  4862551.40999997  |
| 2023-01-03 | 5058973.710000001  |
| 2023-01-04 | 4938142.4699999895 |
| 2023-01-05 | 5002954.230000006  |
| 2023-01-06 | 5020241.789999987  |
| 2023-01-07 | 4974352.960000018  |
| 2023-01-08 | 5020491.3299999945 |
| 2023-01-09 | 5098359.960000016  |
| 2023-01-10 | 5312800.130000001  |
| 2023-01-11 | 5195595.700000019  |
| 2023-01-12 | 5036027.3800000185 |
| 2023-01-13 |  5172407.39999998  |
| 2023-01-14 | 5158187.190000015  |
| 2023-01-15 | 5132430.110000002  |
| 2023-01-16 | 5192341.990000004  |
| 2023-01-17 | 5168015.320000032  |
| 2023-01-18 | 5064907.070000006  |
| 2023-01-19 | 5202390.379999996  |
| 2023-01-20 | 5122747.519999973  |
| 2023-01-21 | 5172218.489999989  |
| 2023-01-22 | 5350576.020000009  |
| 2023-01-23 | 5509342.650000024  |
| 2023-01-24 | 5121699.410000006  |
| 2023-01-25 |     5124780.0

### Шаг 3

(Балл - 0.2) Подготовить дашборд с помощью Dash по пункту 2.f, включив туда графики bar и histogram; вставить в конце ноутбука скрин графиков из дашборда.

In [87]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd
import dash

In [88]:
con_db = sql.connect('Gubanov_transaction.db')
data = pd.read_sql(part_6_request,con_db)


In [84]:
display(data)

,DATE,Total_Ammount
0,2023-01-01,4827656.26
1,2023-01-02,4862551.41
2,2023-01-03,5058973.71
3,2023-01-04,4938142.47
4,2023-01-05,5002954.23
...,...,...
105,2023-04-16,5299386.81
106,2023-04-17,5100973.31
107,2023-04-18,5233557.39
108,2023-04-19,5194846.44


In [102]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

app = Dash()


data = pd.read_sql(part_6_request,con_db)

fig = px.bar(data, x='DATE', y="Total_Ammount")
fig_1 = px.histogram(data, x='DATE', y="Total_Ammount")

fig.update_layout(
xaxis_title="Дата",
yaxis_title="Сумма транзакций"
)

fig_1.update_layout(
xaxis_title="Дата",
yaxis_title="Количество транзакций"
)

app.layout = html.Div(children=[
    html.H1(children='Дашборд: сумма транзакций за каждый день'),

    html.Div(children='''
        Задача: Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).
    '''),

    dcc.Graph(
        id='example-graph_bar',
        figure=fig
        
    ),
     dcc.Graph(
        id='example-graph_hist',
        figure=fig_1
        
    ),
])

app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


### Вставляем изображения полученные на Дашборде

In [103]:
import base64
from IPython.display import HTML

In [121]:
with open("/Users/mikhailgubanov/Yandex.Disk-gubanov.mn@phystech.edu.localized/практика по системам хранения/HW_2/newplot.png", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

In [122]:
HTML('<img src="data:newplot/png;base64,{0}">'.format(encoded_string))


In [124]:
with open("/Users/mikhailgubanov/Yandex.Disk-gubanov.mn@phystech.edu.localized/практика по системам хранения/HW_2/newplot-2.png", "rb") as image_file:
    encoded_string_2 = base64.b64encode(image_file.read()).decode('utf-8')

In [125]:
HTML('<img src="data:newplot/png;base64,{0}">'.format(encoded_string_2))